### Topic 분석 : 잠재의미분석 LSA 

#### 필요한 패키지를 불러온다.

In [ ]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition  import TruncatedSVD

#### 말뭉치.

In [ ]:
my_docs = ["The global warming is becoming more severe",
           "The movie was simply awesome",
           "I like South Asian food",
           "Samsung is announcing a new technology",
           "Machine Learning is an example of awesome technology",
           "All of us were excited at the movie",
           "We have to do more to reverse the global warming"]

In [ ]:
my_docs = [x.lower() for x in my_docs]

#### TF-IDF 생성.
*max_features* : 속성(단어)의 최대 개수. <br>
*min_df* : DF가 이 수치 미만이면 무시. 정수는 개수를 나타내고 실수는 문서의 비율을 의미함. <br> 
*max_df* : DF가 이 수치를 초과하면 무시. 정수는 개수를 나타내고 실수는 문서의 비율을 의미함. stopword를 걸러내려는 목적으로 설정함. <br> 

In [ ]:
my_stop_words = ['us', 'like']

In [ ]:
vectorizer = TfidfVectorizer(max_features = 15, min_df = 1, max_df = 3, stop_words = stopwords.words('english')+my_stop_words)
X = vectorizer.fit_transform(my_docs).toarray()               # Dense Matrix로 변환.

In [ ]:
# X 배열의 크기 (m x n). m=문서의 가짓수 = 7 & n = 단어의 가짓수.
X.shape

In [ ]:
# 단어 출력해 보기.
features = vectorizer.get_feature_names()
print(features)

In [ ]:
# 단어의 가짓수 (n) 출력.
print(len(features))

### LSA 적용 (SVD).

In [ ]:
# 차원 축소.
n_topics = 4
svd = TruncatedSVD(n_components=n_topics, n_iter=100)
svd.fit(X)

In [ ]:
# V^t 행렬 가져오기.
vt = svd.components_
vtabs = np.abs(vt)

In [ ]:
# V^t 행렬의 크기 확인 (r x n)
vt.shape

#### 개개 topic에서 최대 기여 단어 추출.

In [ ]:
n_top = 3
for i in range(n_topics):
    topic_features = [features[idx] for idx in np.argsort(-vt[i,:])]   # argsort는 소 -> 대의 순서로 정렬.
    topic_features_top = topic_features[0:n_top]
    if i == 0:
        topic_matrix = [topic_features_top]                    # list의 list 만들 준비!
    else:
        topic_matrix.append(topic_features_top) 

In [ ]:
# Topic 출력.
topic_matrix

In [ ]:
# Topic 이름 정하기.
topic_names = ['Environment', 'Movie','Technology', 'Cuisine']

#### 개개 문서에서 유력한 topic 인식.

In [ ]:
n_docs = len(my_docs)
for i in range(n_docs):
    score_pick = 0
    topic_pick = 0
    tokennized_doc = nltk.word_tokenize(my_docs[i])
    for j in range(n_topics):
        found = [ x in topic_matrix[j] for x in tokennized_doc ] 
        score = np.sum(found)
        if (score > score_pick):
            score_pick = score
            topic_pick = j
    print("Document " + str(i+1) + " = " + topic_names[topic_pick])